## 기본설정 및 함수정의

In [6]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import unary_union
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

# 아산시청 위도, 경도
Asan = [36.789882248764656, 127.00274491353838]

# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])
    
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

#### 아산시 행정경계 (출처 - 통계지리정보서비스 2023년 센서스용 행정구역경계(읍면동))

In [5]:
# shp to geodataframe convert
shapefile_path = "SBJ_2405_001/_census_data_2023_bnd_dong_bnd_dong_34040_2023_2023/bnd_dong_34040_2023_2023_2Q.shp"
asan_gdf = gpd.read_file(shapefile_path)
asan_gdf = asan_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환

# 행정동 구분에 따라 색 구분하는 함수
def hjd_color(name):
    if name[-1] == '읍':
        return 'green'
    elif name[-1] == '면':
        return 'yellow'
    elif name[-1] == '동':
        return 'red'

asan_gdf['color'] = asan_gdf['ADM_NM'].apply(hjd_color)

#### 격자(매핑용)

In [3]:
# GeoJSON 파일 불러오기
with open('SBJ_2405_001/12.아산시_격자(매핑용).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))
# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')

#### 아산시 상세 도로망

In [4]:
# GeoJSON 파일 불러오기
with open('SBJ_2405_001/4.아산시_상세도로망.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
roadsystem_df = pd.json_normalize(geojson_data['features'])
roadsystem_df['geometry'] = roadsystem_df['geometry.coordinates'].apply(lambda x : make_lin(x))
roadsystem_df['properties.link_id'] = roadsystem_df['properties.link_id'].astype(str)
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadsystem_df = gpd.GeoDataFrame(roadsystem_df, geometry='geometry')

In [ ]:
# map 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

# 통합도로망 추가
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)
grid_layer = folium.GeoJson(
    unary_union(asan_gdf['geometry']),
    name="아산시 행정동",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
)
grid_layer.add_to(m)
roal_geojson = roadsystem_df['geometry'].to_json()
roal_layer = folium.GeoJson(
    roal_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    })
roal_layer.add_to(m)
# 맵 출력
m

#### 아산시 병원진료과목정보

In [8]:
# 병원 데이터프레임을
treatment_df = pd.read_csv('SBJ_2405_001/22.아산시_병원진료과목정보.csv')

In [10]:
# 병원별 진료과목 list mapping한 dict생성
hos_dic = {}
for hos in tqdm(treatment_df['mdcl_inst'].unique()):
    subset = treatment_df[treatment_df['mdcl_inst'] == hos]
    hos_dic[hos] = subset['mdcl_mjr_nm'].tolist()

100%|██████████| 331/331 [00:00<00:00, 3036.76it/s]


#### 아산시 병원정보

In [ ]:
# 병원 데이터프레임을 GeoPandas 데이터프레임으로 변환
hospital_df = pd.read_csv('SBJ_2405_001/21.아산시_병원정보.csv')
hospital_df = geo_transform(hospital_df)
# 병원별 진료과목 mapping
hospital_df['mjr'] = hospital_df['mdcl_inst'].map(hos_dic)
# Update 'mjr' column where it is null with 'mdcl_gbn' column value as a list
hospital_df.loc[hospital_df['mjr'].isnull(), 'mjr'] = hospital_df['mdcl_gbn'].apply(lambda x: [x])


#### 아산시 약국현황

In [ ]:
# 약국 데이터프레임을 GeoPandas 데이터프레임으로 변환
pharmacy_df = pd.read_csv('SBJ_2405_001/23.아산시_약국현황.csv')
pharmacy_df = geo_transform(pharmacy_df)

#### 아산시 AED위치정보

In [ ]:
AED_df = pd.read_csv('SBJ_2405_001/8.아산시_자동심장충격기(AED)위치정보.csv')
AED_df = geo_transform(AED_df)

#### 아산시 119 안전센터

In [ ]:
df_119 = pd.read_csv('SBJ_2405_001/24.아산시_119안전센터정보.csv')
df_119 = geo_transform(df_119)

## 시설위치 시각화

In [ ]:
# 지도 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

# 기본 배경지도를 항상 표시하도록 설정 및 통합도로망 추가
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)
grid_geojson = roadsystem_df['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    }
)
grid_layer.add_to(m)

# 행정구역 polygon 추가
for _, row in asan_gdf.iterrows() :
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 1
        }).add_to(m)

# 병원 위치 mapping
m_hospital = folium.FeatureGroup(name="병원", overlay=True)
for _, row in hospital_df.iterrows() :
    folium.Circle(location=(row['lat'], row['lon']), radius=5, color='red', weight = 3,
            fill='red', name="병원").add_to(m_hospital)
m_hospital.add_to(m)

# 약국 위치 mapping
m_pharmacy = folium.FeatureGroup(name="약국", overlay=True)
for _, row in pharmacy_df.iterrows() :
    folium.Circle(location=(row['lat'], row['lon']), radius=5, color='green', weight = 3,
            fill='red', name="약국").add_to(m_pharmacy)
m_pharmacy.add_to(m)

# AED 위치 매핑
m_AED = folium.FeatureGroup(name="AED", overlay=True)
for _, row in AED_df.iterrows() :
    folium.Circle(location=(row['lat'], row['lon']), radius=5, color='blue', weight = 3,
            fill='red', name="AED").add_to(m_AED)
m_AED.add_to(m)

# 119 위치 매핑
m_119 = folium.FeatureGroup(name="119", overlay=True)
for _, row in df_119.iterrows() :
    folium.Circle(location=(row['lat'], row['lon']), radius=5, color='orange', weight = 5,
            fill='red', name="119").add_to(m_119)
m_119.add_to(m)

# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 지도 출력
m

In [22]:
m.save("visualization/4. 아산시 의료불균형 분석/아산시 의료시설위치 시각화.html")